In [ ]:
import tensorflow as tf
import random
import os
import pickle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

seed_everything(42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/캡스톤/data/Anime3/'

In [ ]:
with open(path+"train_id_data.pickle","rb") as fr:
    train_data = pickle.load(fr)
    
with open(path+"valid_id_data.pickle","rb") as fr:
    valid_data = pickle.load(fr)
    
with open(path+"test_id_data.pickle","rb") as fr:
    test_data = pickle.load(fr)

In [ ]:
with open(path+"train_target.pickle","rb") as fr:
    train_target = pickle.load(fr)
    
with open(path+"valid_target.pickle","rb") as fr:
    valid_target = pickle.load(fr)
    
with open(path+"test_target.pickle","rb") as fr:
    test_target = pickle.load(fr)

In [ ]:
test_data[0]

array([[[    1],
        [33486],
        [36456]],

       [[    1],
        [33486],
        [36456]],

       [[    1],
        [33486],
        [36456]],

       ...,

       [[34134],
        [27899],
        [37779]],

       [[34134],
        [27899],
        [37779]],

       [[34134],
        [27899],
        [37779]]])

## Model

In [ ]:
from tensorflow import keras
from keras.layers import Input, Embedding, LSTM, Dense, Lambda, Multiply, Bidirectional, Flatten, BatchNormalization, Add
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPool1D
from keras.models import Model
from keras.layers.wrappers import TimeDistributed
import numpy as np
import keras.backend as K
from random import randint

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
description_num = 3
#current_path ='data/'
#local_path = 'data/'

In [ ]:
# set parameters:
# V = 142381
# V = 226928
V = 48456  #33531 # len(tokenizer.word_index)
embedding_dim = 50
max_len =  1   # 1945  # maxlen
filters_num = 64
kernel_size = 3

In [ ]:
test_data[0][0]

array([[    1],
       [33486],
       [36456]])

In [ ]:
# description_input=Input(shape=(10,50))
# description_validation=Input(shape=(50,))

input_1 = Input(shape=(description_num, max_len))
input_2 = Input(shape=(max_len,))
embedding = Embedding(input_dim=V,
                      output_dim=embedding_dim,
                      input_length=max_len)
conv1d = Conv1D(filters=filters_num,
                kernel_size=kernel_size,
                activation='relu',
                padding='same')
maxpool1d = MaxPool1D(max_len)
dense = Dense(15)

# input_vector=TimeDistributed(embedding)(description_input)
# validation_vector=embedding(description_validation)

input_vector = TimeDistributed(embedding)(input_1)
validation_vector = embedding(input_2)

convolutional_vector = TimeDistributed(conv1d)(input_vector)
validation_conv = conv1d(validation_vector)

maxpooling_vector = TimeDistributed(maxpool1d)(convolutional_vector)
validation_maxpooling = maxpool1d(validation_conv)

middle_output = TimeDistributed(dense)(maxpooling_vector)
middle_validation = dense(validation_maxpooling)

In [ ]:
print(input_vector.shape, validation_vector.shape,
      convolutional_vector.shape, validation_conv.shape,
      maxpooling_vector.shape, validation_maxpooling.shape,
      middle_output.shape, middle_validation.shape)

(None, 3, 1, 50) (None, 1, 50) (None, 3, 1, 64) (None, 1, 64) (None, 3, 1, 64) (None, 1, 64) (None, 3, 1, 15) (None, 1, 15)


In [ ]:
def change_dim_1(X):
    return K.squeeze(X, 1)

def change_dim_2(X):
    return K.squeeze(X, 2)

def repeat(X):
    return K.repeat_elements(X, description_num, 1)

def repeat1(X):
    return K.repeat_elements(X, 15, 2)

def repeat2(X):
    return K.repeat_elements(X, description_num, 1)

def dot(X, Y):
    return K.dot(X, Y)

def sum_item(X):
    return K.sum(X, axis=2)

def sqrt_item(X):
    return K.sqrt(X)

def cal_denominator(X):
    return 1/(X)

def expand_item(X):
    return K.expand_dims(X, 2)

def expand_rate(X):
    return K.expand_dims(X, 1)

def sum_user(X):
    return K.sum(X, 1)

def sum_rate(X):
    return K.sum(X, 1)

def single_exp(X):
    return K.exp(X)

def sum_exp_denominator(X):
    return 1/(K.sum(K.exp(X), 1))

In [ ]:
# def _test_generator(filename):

middle_output_final = Lambda(change_dim_2)(middle_output)#?*10*15
middle_validation_final = Lambda(repeat)(middle_validation)#?*10*15
molecule = Multiply()([middle_output_final, middle_validation_final])
molecule = Lambda(sum_item)(molecule)

denominator1 = Multiply()([middle_output_final, middle_output_final])
denominator1 = Lambda(sum_item)(denominator1)
denominator1 = Lambda(sqrt_item)(denominator1)

denominator2 = Multiply()([middle_validation_final, middle_validation_final])
denominator2 = Lambda(sum_item)(denominator2)
denominator2 = Lambda(sqrt_item)(denominator2)


denominator = Multiply()([denominator1, denominator2])
denominator = Lambda(cal_denominator)(denominator)

similarity = Multiply()([molecule, denominator])
similarity = Lambda(expand_item)(similarity)
similarity = Lambda(repeat1)(similarity)


user = Multiply()([similarity, middle_output_final])
user = Lambda(sum_user)(user)


item = Lambda(change_dim_1)(middle_validation)


rate_hat = Multiply()([user, item])
rate_hat = Lambda(sum_rate)(rate_hat)
rate_hat = Lambda(expand_rate)(rate_hat)

In [ ]:
model = Model(inputs=[input_1, input_2], outputs=rate_hat)
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae', 'mape'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3, 1)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 time_distributed (TimeDistribu  (None, 3, 1, 50)    2422800     ['input_1[0][0]']                
 ted)                                                                                             
                                                                                                  
 embedding (Embedding)          (None, 1, 50)        2422800     ['input_2[0][0]']            

In [ ]:
hist = model.fit(train_data, train_target,
                 validation_data=(valid_data, valid_target),
                 steps_per_epoch=256,
                 validation_steps=256,
                 epochs=30,
                 verbose=1)

Epoch 1/30
256/256 [==============================] - 16s 10ms/step - loss: 12.0991 - mse: 12.0991 - mae: 2.2746 - mape: 30.5906 - val_loss: 2.4428 - val_mse: 2.4428 - val_mae: 1.1658 - val_mape: 20.4012
Epoch 2/30
256/256 [==============================] - 2s 8ms/step - loss: 2.0904 - mse: 2.0904 - mae: 1.0976 - mape: 18.2061 - val_loss: 2.3844 - val_mse: 2.3844 - val_mae: 1.1642 - val_mape: 20.1528
Epoch 3/30
256/256 [==============================] - 2s 8ms/step - loss: 2.0267 - mse: 2.0267 - mae: 1.0793 - mape: 17.8550 - val_loss: 2.4407 - val_mse: 2.4407 - val_mae: 1.1496 - val_mape: 20.2517
Epoch 4/30
256/256 [==============================] - 2s 8ms/step - loss: 1.9961 - mse: 1.9961 - mae: 1.0714 - mape: 17.6141 - val_loss: 2.4035 - val_mse: 2.4035 - val_mae: 1.1477 - val_mape: 20.1507
Epoch 5/30
256/256 [==============================] - 2s 9ms/step - loss: 1.9784 - mse: 1.9784 - mae: 1.0657 - mape: 17.4828 - val_loss: 2.4110 - val_mse: 2.4110 - val_mae: 1.1391 - val_mape: 20.3

In [ ]:
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_5s.txt','test_whole_list_5s.txt'),steps=134103)
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_10.txt','test_whole_list_10.txt'),steps=34266)
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_6s.txt','test_whole_list_6s.txt'),steps=134103)
scores = model.evaluate(test_data, test_target, steps=2500)
print(scores)

2500/2500 [==============================] - 7s 3ms/step - loss: 2.6855 - mse: 2.6855 - mae: 1.2284 - mape: 20.1972
[2.6854591369628906, 2.685459613800049, 1.2284021377563477, 20.19718360900879]


In [ ]:
#model.save(path+"IMDB_ADLFM_baseline")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/캡스톤/data/IMDB_ADLFM_baseline/assets
